In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
api_key = os.environ["GRAPHRAG_API_KEY"]
llm_model = os.environ["GRAPHRAG_LLM_MODEL"]
embedding_model = os.environ["GRAPHRAG_EMBEDDING_MODEL"]
api_base = os.environ["GRAPHRAG_API_BASE"]
api_version = os.environ["GRAPHRAG_API_VERSION"]

llm = ChatOpenAI(
    api_key=api_key,
    api_base=api_base,
    api_version=api_version,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)

token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "../graphfleet/output/20240826-060116/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

Total report count: 110
Report count after filtering by community level 2: 90


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,100,# MIRAGE Datasets and Their Evaluation in NLP\...,2,9.0,MIRAGE Datasets and Their Evaluation in NLP,The MIRAGE datasets and their associated evalu...,The community revolves around the MIRAGE datas...,[{'explanation': 'The MIRAGE datasets serve as...,"{\n ""title"": ""MIRAGE Datasets and Their Eva...",1130a53b-d692-49b6-a573-13d5bcf45335
1,101,# Graph RAG and Community Summaries in NLP\n\n...,2,9.0,Graph RAG and Community Summaries in NLP,The text is highly relevant and significant to...,The community focuses on the use of Graph Retr...,[{'explanation': 'Community summaries are comp...,"{\n ""title"": ""Graph RAG and Community Summa...",eac2c84e-2c96-4462-a427-657a909225dd
2,102,# Advancements in Computational Linguistics an...,2,9.0,Advancements in Computational Linguistics and NLP,The text is highly relevant and impactful in a...,The community of Computational Linguistics and...,[{'explanation': 'Recent advancements in retri...,"{\n ""title"": ""Advancements in Computational...",839c3851-2561-4c60-be78-d1bdc38691ea
3,103,# Advancements in Computational Linguistics an...,2,9.0,Advancements in Computational Linguistics and NLP,The text is highly relevant and impactful in a...,The community of Computational Linguistics and...,[{'explanation': 'GLOBAL SUMMARIZATION is a me...,"{\n ""title"": ""Advancements in Computational...",b88a3c23-e191-483d-8451-e94622d6fe92
4,104,# Advancements in Graph-Based Retrieval-Augmen...,2,9.0,Advancements in Graph-Based Retrieval-Augmente...,The rating is high due to the significant impa...,The community focuses on the development and a...,[{'explanation': 'The GRAPH INDEX is an advanc...,"{\n ""title"": ""Advancements in Graph-Based R...",39ce50ff-6b80-4a28-83d2-8e7ca17f035e


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=True,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [8]:
result = await search_engine.asearch(
    "Show me an agent that can help me create a knowledge graph"
)

print(result.response)

### Creating a Knowledge Graph with AgentInstruct

If you are looking to create a knowledge graph, **AgentInstruct** is a highly effective tool designed for this purpose. AgentInstruct leverages agentic flows to automate the data generation process, significantly enhancing the efficiency and effectiveness of constructing knowledge graphs. This system is particularly useful for generating high-quality synthetic data at scale, which is crucial for improving the performance and robustness of language models [Data: Reports (23, 94, 96, 56)].

#### Key Features of AgentInstruct

1. **Automated Data Generation**: AgentInstruct automates the creation of synthetic data, reducing the need for extensive human expertise. This automation is pivotal in generating diverse and challenging datasets that enhance the reading comprehension capabilities of language models like Mistral [Data: Reports (94)].

2. **Enhanced Knowledge Graph Construction**: By leveraging agentic flows, AgentInstruct facilitate

In [10]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,41,Graph RAG and its Contributors,0.600000,# Graph RAG and its Contributors\n\nThe commun...,9.5
1,64,Advancements in Large Language Models and Thei...,0.333333,# Advancements in Large Language Models and Th...,9.0
2,76,Advancements in Retrieval-Augmented Generation...,0.333333,# Advancements in Retrieval-Augmented Generati...,9.0
3,19,Advancements in Retrieval-Augmented Generation...,0.200000,# Advancements in Retrieval-Augmented Generati...,9.0
4,104,Advancements in Graph-Based Retrieval-Augmente...,0.200000,# Advancements in Graph-Based Retrieval-Augmen...,9.0
...,...,...,...,...,...
85,103,Advancements in Computational Linguistics and NLP,0.200000,# Advancements in Computational Linguistics an...,9.0
86,75,Advancements in Computational Linguistics and NLP,0.133333,# Advancements in Computational Linguistics an...,9.0
87,108,Advancements in Retrieval-Augmented Generation...,0.133333,# Advancements in Retrieval-Augmented Generati...,9.0
88,50,Advancements in Retrieval-Augmented Generation...,0.066667,# Advancements in Retrieval-Augmented Generati...,9.0


In [11]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 8. LLM tokens: 85934
